In [ ]:
// Nuget Packages
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.*"

#r "nuget: Microsoft.SemanticKernel, 1.*"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.*-*"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.*-*"

In [ ]:
// Add using statements
using System.ComponentModel;
using System.IO;
using System.Text.Json;
using System.Text.Json.Nodes;

using Microsoft.Extensions.Configuration;

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.SemanticKernel.Plugins.Core;


In [ ]:
// Azure OpenAI configurations
var path = Path.Combine(Directory.GetCurrentDirectory(), "appsettings.Development.json");
var config = new ConfigurationBuilder().AddJsonFile(path).Build();

var endpoint = config["Azure:OpenAI:Endpoint"];
var apiKey = config["Azure:OpenAI:ApiKey"];
var deploymentName = config["Azure:OpenAI:DeploymentName"];

In [ ]:
// Build Semantic Kernel
var kernel = Kernel.CreateBuilder()
                   .AddAzureOpenAIChatCompletion(
                       deploymentName: deploymentName,
                       endpoint: endpoint,
                       apiKey: apiKey)
                   .Build();

In [ ]:
// Plugin: Music Library
public class MusicLibraryPlugin
{
    [KernelFunction, Description("Get a list of music recently played by the user")]
    public static string GetRecentPlays()
    {
        string content = File.ReadAllText("data/recentlyplayed.json");
        return content;
    }

    [KernelFunction, Description("Add a song to the recently played list")]
    public static string AddToRecentlyPlayed(
        [Description("The name of the artist")] string artist, 
        [Description("The title of the song")] string song, 
        [Description("The song genre")] string genre)
    {
        // Read the existing content from the file
        string jsonContent = File.ReadAllText("data/recentlyplayed.json");
        var recentlyPlayed = (JsonArray) JsonNode.Parse(jsonContent);

        var newSong = new JsonObject
        {
            ["title"] = song,
            ["artist"] = artist,
            ["genre"] = genre
        };

        recentlyPlayed.Insert(0, newSong);
        File.WriteAllText("data/recentlyplayed.json",
            JsonSerializer.Serialize(
                recentlyPlayed,
                new JsonSerializerOptions { WriteIndented = true }));

        return $"Added '{song}' to recently played";
    }
}

In [ ]:
// Plugin: Music Concert
public class MusicConcertPlugin
{
    [KernelFunction, Description("Get a list of upcoming concerts")]
    public static string GetTours()
    {
        string content = File.ReadAllText("data/concertdates.json");
        return content;
    }
}

In [ ]:
// Import plugins and prompts
kernel.ImportPluginFromType<MusicLibraryPlugin>();
kernel.ImportPluginFromType<MusicConcertPlugin>();
kernel.ImportPluginFromPromptDirectory("Prompts/Concert");

In [ ]:
// Define the goal
string location = "San Francisco, CA, USA";
string goal = @$"""
    Based on the user's recently played music, suggest a concert for the user living in ${location}
""";

In [ ]:
// Initialise planner
#pragma warning disable SKEXP0060
var planner = new HandlebarsPlanner(new HandlebarsPlannerOptions() { AllowLoops = true });
#pragma warning restore SKEXP0060


In [ ]:
// Run the planner and get the result
#pragma warning disable SKEXP0060
var plan = await planner.CreatePlanAsync(kernel, goal);
#pragma warning restore SKEXP0060

#pragma warning disable SKEXP0060
var result = await plan.InvokeAsync(kernel);
#pragma warning restore SKEXP0060

Console.WriteLine($"Results: {result}");

In [ ]:
// Get plan
#pragma warning disable SKEXP0060
var plan = await planner.CreatePlanAsync(kernel, goal);
#pragma warning restore SKEXP0060

Console.WriteLine("Concert Plan:");
Console.WriteLine(plan);


In [ ]:
// Create another function
var songSuggesterFunction = kernel.CreateFunctionFromPrompt(
    promptTemplate: @"Based on the user's recently played music:
        {{$recentlyPlayedSongs}}
        recommend a song to the user from the music library:
        {{$musicLibrary}}",
    functionName: "SuggestSong",
    description: "Suggest a song to the user"
);

In [ ]:
// Add function
kernel.Plugins.AddFromFunctions("SuggestSongPlugin", [songSuggesterFunction]);

In [ ]:
// Create another plan
#pragma warning disable SKEXP0060
var songSuggestPlan = await planner.CreatePlanAsync(kernel, @"Suggest a song from the music library to the user based on their recently played songs");
#pragma warning restore SKEXP0060

Console.WriteLine("Song Plan:");
Console.WriteLine(songSuggestPlan);

In [ ]:
// Add prompt execution settings for auto invocation
OpenAIPromptExecutionSettings settings = new()
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

In [ ]:
// New prompt
string prompt = """
    I live in Portland OR USA.
    Based on my recently played songs and a list of upcoming concerts,
    which concert do you recommend?
""";

In [ ]:
// Get the result from the auto invocation
var result = await kernel.InvokePromptAsync(prompt, new KernelArguments(settings));

Console.WriteLine(result);